In [ ]:
import os
import requests
from tqdm import tqdm

# Cookie for authentication
COOKIE = {
    "csrftoken": "I9x2jvGGE4se3MBa9moavDtC9o8YEgaA4Rup5ijhHJjCTRn0qRpHGJW06XG0SooG",
    "sessionid": "y44nlmh7rjrqvvxj902jc8pmw918m1p7",
}

# Base URL for file downloads
BASE_URL = "https://api.mosaiks.org/portal/download_grid_file/coarsened_global_dense_grid_decimal_place=1_GHS_pop_weight=True"

# Destination directory
DEST_DIR = "/Users/isaiah/code/ai4good/data/wk5/mosaiks001"
os.makedirs(DEST_DIR, exist_ok=True)  # Ensure the directory exists

# Regions and chunks to download
regions = {
    # "Africa": [1, 2, 3],
    # "Asia": [1, 2, 3, 4, 5, 6],
    # "Europe": [1, 2],
    # "North America": [1, 2, 3],
    "Oceania": [1],
    # "South America": [1, 2],
    "Australia": [1],
}

# Function to download a file with progress bar
def download_file(url, filename):
    response = requests.get(url, cookies=COOKIE, stream=True, verify=False)
    
    if response.status_code == 200:
        total_size = int(response.headers.get("content-length", 0))
        with open(filename, "wb") as f, tqdm(
            desc=os.path.basename(filename),
            total=total_size,
            unit="B",
            unit_scale=True,
            unit_divisor=1024,
        ) as bar:
            for chunk in response.iter_content(1024):
                f.write(chunk)
                bar.update(len(chunk))
        print(f"✅ Downloaded: {filename}")
    else:
        print(f"❌ Failed to download {url} (Status {response.status_code})")

# Download each file
for region, chunks in regions.items():
    for chunk in chunks:
        filename = os.path.join(DEST_DIR, f"{region.lower()}_{chunk}.zip")
        url = f"{BASE_URL}/coarsened_global_dense_grid_decimal_place=1_GHS_pop_weight=True_{region}_chunk={chunk}.zip"
        if (region == "Australia" or region == "Oceania"):
            url = f"{BASE_URL}/coarsened_global_dense_grid_decimal_place=1_GHS_pop_weight=True_{region}.zip"
        
        print(f"📥 Downloading {url}...")
        download_file(url, filename)


In [ ]:
# unzip the files
import zipfile
import os
import glob

# Define the directory containing the zip files
zip_dir = "/Users/isaiah/code/ai4good/data/wk5/mosaiks100"
# Define the directory to extract the contents
extract_dir = "/Users/isaiah/code/ai4good/data/wk5/mosaiks100/"
# Create the extraction directory if it doesn't exist
os.makedirs(extract_dir, exist_ok=True)
# Loop through all zip files in the directory
for zip_file in glob.glob(os.path.join(zip_dir, "*.zip")):
    # Open the zip file
    with zipfile.ZipFile(zip_file, "r") as z:
        # Extract all the contents into the extraction directory
        z.extractall(extract_dir)
        print(f"Extracted {zip_file} to {extract_dir}")


In [ ]:
# combine the csv files into one for each continent
import pandas as pd
import os
import glob

# Define the directory containing the CSV files
csv_dir = "/Users/isaiah/code/ai4good/data/wk5/mosaiks001/"
# Define the output directory for the combined CSV files
output_dir = "/Users/isaiah/code/ai4good/data/wk5/mosaiks001/"
# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)
# Loop through each continent
for continent in ["Europe", "North_America", "South_America", "Oceania", "Asia"]:
    print(f"Combining {continent} files...")
    # Find all CSV files for the continent
    csv_files = glob.glob(os.path.join(csv_dir, f"*{continent}_*.csv"))
    # Combine the CSV files into one DataFrame
    print(f"Combining {len(csv_files)} files...")
    combined_df = pd.concat([pd.read_csv(f) for f in csv_files], ignore_index=True)
    # Save the combined DataFrame to a new CSV file
    combined_df.to_csv(os.path.join(output_dir, f"{continent}.csv"), index=False)
    print(f"Combined {len(csv_files)} files into {continent}.csv")